In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
from google.colab import files

uploaded = files.upload()

Saving Brooklyn_bridge.txt to Brooklyn_bridge.txt


In [0]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

In [70]:
tokenizer = Tokenizer(num_words=10000)

data = open('Brooklyn_bridge.txt',encoding="UTF-8").read()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)

input_sequences = []

for text in corpus:

  encoded_seq = tokenizer.texts_to_sequences([text])[0]

  for i in range(1,len(encoded_seq)):

    temp = encoded_seq[:i+1]

    input_sequences.append(temp)

max_len = max([len(x) for x in input_sequences])

padded_seq = np.array(pad_sequences(input_sequences,maxlen=max_len,padding='pre'))

X = padded_seq[:,:-1]

Y = padded_seq[:,-1]

total_words = len(tokenizer.word_index)+1


Y = ku.to_categorical(Y,num_classes=total_words)

print(X)



[[   0    0    0 ...    0    0  539]
 [   0    0    0 ...    0    0  540]
 [   0    0    0 ...    0  540  121]
 ...
 [   0    0    0 ...    0  353    2]
 [   0    0    0 ...  353    2 1673]
 [   0    0    0 ...    2 1673    3]]


In [72]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 11, 100)           167400    
_________________________________________________________________
bidirectional (Bidirectional (None, 11, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 11, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 837)               84537     
_________________________________________________________________
dense_1 (Dense)              (None, 1674)              1402812   
Total params: 2,116,349
Trainable params: 2,116,349
Non-trainable params: 0
____________________________________________

In [75]:
history = model.fit(X, Y, epochs=200, verbose=1)

Epoch 1/200
121/121 [==============================] - 1s 10ms/step - loss: 4.9373 - accuracy: 0.1102
Epoch 2/200
121/121 [==============================] - 1s 10ms/step - loss: 4.8837 - accuracy: 0.1128
Epoch 3/200
121/121 [==============================] - 1s 10ms/step - loss: 4.8423 - accuracy: 0.1138
Epoch 4/200
121/121 [==============================] - 1s 10ms/step - loss: 4.7978 - accuracy: 0.1133
Epoch 5/200
121/121 [==============================] - 1s 10ms/step - loss: 4.7427 - accuracy: 0.1213
Epoch 6/200
121/121 [==============================] - 1s 10ms/step - loss: 4.6998 - accuracy: 0.1252
Epoch 7/200
121/121 [==============================] - 1s 10ms/step - loss: 4.6457 - accuracy: 0.1265
Epoch 8/200
121/121 [==============================] - 1s 10ms/step - loss: 4.6032 - accuracy: 0.1311
Epoch 9/200
121/121 [==============================] - 1s 9ms/step - loss: 4.5505 - accuracy: 0.1394
Epoch 10/200
121/121 [==============================] - 1s 10ms/step - loss: 4.5146

In [88]:
seed_text = "what is life"

num = 97

for _ in range(num):

  encoded = tokenizer.texts_to_sequences([seed_text])[0]

  padded = pad_sequences([encoded],maxlen=max_len-1,padding='pre')

  output_index = np.argmax(model.predict(padded)[0])

  for word,index in tokenizer.word_index.items():

    if output_index == index:

      seed_text += " " + word

word_list = seed_text.split(' ')

poem_list = []

for i in range(5,len(word_list),5):

  line = word_list[i-4:i]

  str_line = ' '.join(line)

  poem_list.append(str_line)


final_poem = '\n'.join(poem_list)

print(final_poem)









is life the sun
world weeping real the
bottom side of never
came i fall for
hold to the day
blue enough to crack
land you used to
things never grown never
friends when the moon
heart meant to be
so high are i
new fall its to
day i came the
so high not i
with a thread may
one to the man
on face in love
i know by my
avoided call that times
